In [7]:
#Install what is needed
!pip install pandas numpy matplotlib xarray hvplot
!pip install rasterio
!pip install rioxarray
!pip install fiona 
!pip install geopandas
!pip install cartopy
!pip install geopandas geoviews
!pip install netCDF4
!pip install pydap
!pip install pygbif
!pip install osmnx
!pip install earthaccess


#Import from installations
import fiona
import os
import pathlib
import hvplot.xarray  
import rioxarray  
import pathlib
import requests
import zipfile
import geopandas as gpd 
import hvplot.pandas
import netCDF4
import earthaccess


import xarray as xr
import geopandas as gpd
import rioxarray as rxr
import matplotlib.pyplot as plt
import geopandas as gpd
import cartopy.crs as ccrsp
import pandas as pd
import numpy as np


from math import floor, ceil
from shapely.geometry import box
from rioxarray.merge import merge_arrays
from osmnx import features as osm
from pygbif import occurrences
from glob import glob
from rioxarray.merge import merge_arrays
from shapely.geometry import Point


In [8]:
iplant_user = os.environ['IPLANT_USER']
data_path = (
    pathlib.Path.home()
    / 'data-store' / 'home' / iplant_user / 'habitat_suitability')

data_path.mkdir(parents = True, exist_ok = True)

data_path

PosixPath('/home/jovyan/data-store/home/elemilene01/habitat_suitability')

In [9]:
# Define and create the project data directory
habitat_suitability_data_dir = os.path.join(
    pathlib.Path.home(),
    'earth-analytics',
    'data',
    'habitat_suitability'
)
os.makedirs(habitat_suitability_data_dir, exist_ok=True)

# Call the data directory to check its location
habitat_suitability_data_dir


# Define correct project directory path
project_data_dir = os.path.join(
    pathlib.Path.home(),
    'Rocky_Mountain_Juniper_Study',
    'data',
    'habitat_suitability'
)
os.makedirs(project_data_dir, exist_ok=True)

print("Data directory is:", project_data_dir)

# Define WKT bounding box for ROMO area
bbox = "POLYGON((-105.9 40.1, -105.4 40.1, -105.4 40.6, -105.9 40.6, -105.9 40.1))"

# Query GBIF for Juniperus scopulorum
records = occurrences.search(
    scientificName="Juniperus scopulorum",
    hasCoordinate=True,
    limit=1000,
    geometry=bbox
)

# Preview first few results
for rec in records['results'][:5]:
    print(rec.get('species'), rec.get('decimalLatitude'), rec.get('decimalLongitude'))

Data directory is: /home/jovyan/Rocky_Mountain_Juniper_Study/data/habitat_suitability
Juniperus scopulorum 40.377706 -105.542436
Juniperus scopulorum 40.436258 -105.551189
Juniperus scopulorum 40.473475 -105.444317
Juniperus scopulorum 40.472766 -105.443738
Juniperus scopulorum 40.374133 -105.571019


In [10]:
# Search Juniperus scopulorum within bounding box (ROMO)
forest_gdf = osm.features_from_place(
    query = 'Roosevelt National Forest',
    tags = {'name':'Roosevelt National Forest'})

forest_gdf

type        boundary                       name  \
element  id                                                            
relation 395767  boundary  protected_area  Roosevelt National Forest   
         395767  boundary  protected_area  Roosevelt National Forest   
         395767  boundary  protected_area  Roosevelt National Forest   
         395767  boundary  protected_area  Roosevelt National Forest   
         395767  boundary  protected_area  Roosevelt National Forest   
         395767  boundary  protected_area  Roosevelt National Forest   

                                     operator operator:short operator:type  \
element  id                                                                  
relation 395767  United States Forest Service           USFS        public   
         395767  United States Forest Service           USFS        public   
         395767  United States Forest Service           USFS        public   
         395767  United States Forest Service           USFS        public   
         395767  United States Forest Service           USFS        public   
         395767  United States Forest Service           USFS        public   

                operator:wikidata               operator:wikipedia ownership  \
element  id                                                                    
relation 395767          Q1891156  en:United States Forest Service  national   
         395767          Q1891156  en:United States Forest Service  national   
         395767          Q1891156  en:United States Forest Service  national   
         395767          Q1891156  en:United States Forest Service  national   
         395767          Q1891156  en:United States Forest Service  national   
         395767          Q1891156  en:United States Forest Service  national   

                protect_class   protected  protected_area protection_title  \
element  id                                                                  
relation 395767             6  perpetuity  forest_reserve  National Forest   
         395767             6  perpetuity  forest_reserve  National Forest   
         395767             6  perpetuity  forest_reserve  National Forest   
         395767             6  perpetuity  forest_reserve  National Forest   
         395767             6  perpetuity  forest_reserve  National Forest   
         395767             6  perpetuity  forest_reserve  National Forest   

                                     website  wikidata  \
element  id                                              
relation 395767  https://www.fs.usda.gov/arp  Q7366477   
         395767  https://www.fs.usda.gov/arp  Q7366477   
         395767  https://www.fs.usda.gov/arp  Q7366477   
         395767  https://www.fs.usda.gov/arp  Q7366477   
         395767  https://www.fs.usda.gov/arp  Q7366477   
         395767  https://www.fs.usda.gov/arp  Q7366477   

                                    wikipedia  \
element  id                                     
relation 395767  en:Roosevelt National Forest   
         395767  en:Roosevelt National Forest   
         395767  en:Roosevelt National Forest   
         395767  en:Roosevelt National Forest   
         395767  en:Roosevelt National Forest   
         395767  en:Roosevelt National Forest   

                                                          geometry  
element  id                                                         
relation 395767  MULTIPOLYGON (((-106.15924 40.99745, -106.1445...  
         395767  MULTIPOLYGON (((-106.15924 40.99745, -106.1445...  
         395767  MULTIPOLYGON (((-106.15924 40.99745, -106.1445...  
         395767  MULTIPOLYGON (((-106.15924 40.99745, -106.1445...  
         395767  MULTIPOLYGON (((-106.15924 40.99745, -106.1445...  
         395767  MULTIPOLYGON (((-106.15924 40.99745, -106.1445...

In [11]:
# Convert GBIF records (list of dicts) to DataFrame
juniper_gdf = gpd.read_file ('Juniper_Occur.zip')

juniper_gdf.eventDate = pd.to_datetime(juniper_gdf.eventDate , format='ISO8601', utc=True)

# Drop rows without lat/lon
juniper_gdf = juniper_gdf.dropna(subset=["decimalLatitude", "decimalLongitude","eventDate"])

juniper_gdf = (
    gpd.GeoDataFrame(
        juniper_gdf, 
        geometry=gpd.points_from_xy(
            juniper_gdf.decimalLongitude, 
            juniper_gdf.decimalLatitude), 
        crs="EPSG:4326")
    # Select the desired columns
    [['eventDate','geometry']]
)
juniper_gdf

,eventDate,geometry
0,2013-11-30 15:15:40+00:00,POINT (-105.37904 39.65909)
1,2011-05-19 00:00:00+00:00,POINT (-103.30006 37.21794)
2,2011-05-19 00:00:00+00:00,POINT (-103.30006 37.21794)
3,2011-05-19 00:00:00+00:00,POINT (-102.98788 37.09172)
4,2008-10-22 00:00:00+00:00,POINT (-105.20178 39.6553)
...,...,...
1692,2017-03-18 10:17:00+00:00,POINT (-105.3013 39.9977)
1693,2003-05-03 00:00:00+00:00,POINT (-103.99388 40.81583)
1694,2003-03-25 00:00:00+00:00,POINT (-104.96555 39.74555)
1695,2003-03-16 00:00:00+00:00,POINT (-105.11111 40.755)


In [12]:
# Ensure both are in EPSG:4326 for web maps
forest_gdf = forest_gdf.to_crs("EPSG:4326")
juniper_gdf = juniper_gdf.to_crs("EPSG:4326")

juniper_occur_gdf = juniper_gdf.sjoin(
    gpd.GeoDataFrame(
        geometry = forest_gdf.envelope),
    predicate = 'within')

juniper_occur_gdf

,eventDate,geometry,element,id
5,2008-10-22 00:00:00+00:00,POINT (-105.30332 40.00425),relation,395767
5,2008-10-22 00:00:00+00:00,POINT (-105.30332 40.00425),relation,395767
5,2008-10-22 00:00:00+00:00,POINT (-105.30332 40.00425),relation,395767
5,2008-10-22 00:00:00+00:00,POINT (-105.30332 40.00425),relation,395767
5,2008-10-22 00:00:00+00:00,POINT (-105.30332 40.00425),relation,395767
...,...,...,...,...
1696,2016-01-24 11:04:51+00:00,POINT (-105.28881 39.92924),relation,395767
1696,2016-01-24 11:04:51+00:00,POINT (-105.28881 39.92924),relation,395767
1696,2016-01-24 11:04:51+00:00,POINT (-105.28881 39.92924),relation,395767
1696,2016-01-24 11:04:51+00:00,POINT (-105.28881 39.92924),relation,395767


In [13]:
# ROMO boundary plot 
forest_plot = forest_gdf.hvplot(
    geo=True,
    line_color='orange',
    line_width=4,
    alpha=0.7,
    tiles='EsriImagery',  
    frame_width=800,
    frame_height=600,
    title="Juniperus scopulorum in Roosevelt National Forest"
)

# GBIF points 
juniper_plot = juniper_occur_gdf.hvplot.points(
    geo=True,
    color='white',
    size=10,
    alpha=0.6
)

# Combine
forest_plot * juniper_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]
   .Points.I   :Points   [Longitude,Latitude]